This notebook is in a DRAFT state and will be updated soon

# Helper functions and classes

In [1]:
# testers.py

import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import clear_output

# from logger import print_shifted_permutation
# from profile_solver import ProfileInversionsSolver


def L(a):
    result = a[1:]
    result.append(a[0])
    return result

def R(a):
    result = [a[-1]]
    result.extend(a[:-1])
    return result

def X(a):
    result = [a[1], a[0]]
    result.extend(a[2:])
    return result

ops = {
    'L': L,
    'R': R,
    'X': X,
}

def apply_solution(perm, solution):
    result = perm
    for op in solution:
        result = ops[op](result)

    return result

def check_solution(perm, solution):
    n = len(perm)
    sorted_perm = [x for x in range(n)]
    return apply_solution(perm.copy(), solution) == sorted_perm

def test(n, solver, print_diff=False, **kwargs):
    score = 0
    for i in range(2, n + 1):
        sorted_perm = [x for x in range(i)]
        perm = R(R(sorted_perm.copy()[::-1]))
        moves = solver(perm.copy(), **kwargs).solve()
        score += len(moves)
        result = apply_solution(perm.copy(), moves)

        if result != sorted_perm:
            print('WA at n={}, got {}'.format(i, result))
        if print_diff and len(moves) != i * (i - 1) / 2:
            print(i, len(moves), len(moves) - i * (i - 1) / 2)

    print('Total score: ', score)

def test_random(n, count, solver, **kwargs):
    total_score = 0
    unerrored_score = 0
    errors_count = 0
    scores = []
    for _ in range(count):
        sorted_perm = [x for x in range(n)]
        perm = np.random.permutation(n).tolist()
        moves = solver(perm.copy(), **kwargs).solve()
        result = apply_solution(perm, moves)

        total_score += len(moves)
        if result != sorted_perm:
            errors_count += 1
            print('WA at n={}, got {}, on perm={}'.format(n, result, perm))
            print(len(moves))
        else:
            unerrored_score += len(moves)
            scores.append(len(moves))

    # print('Total errors: {}/{}'.format(errors_count, count))
    print('Average score: {}'.format(total_score / count))
    # print('Unerrored score: {}'.format(unerrored_score / (count - errors_count)))

    return scores

def visualize_solution(perm, solution, start_pos=0):
    result = perm
    pos = start_pos
    n = len(perm)

    print(' ' * 3, end='')
    print_shifted_permutation(result, pos, print_func=print)
    for op in solution:
        result = ops[op](result)
        if op == 'R':
            pos = (pos - 1 + n) % n
        elif op == 'L':
            pos = (pos + 1) % n

        print(op + ': ', end='')
        print_shifted_permutation(result, pos, print_func=print)

    print()

def visualize_solution_at_n(n, solver, start_pos=0, perm=None, solver_kwargs=dict()):
    sorted_perm = [x for x in range(n)]
    perm = perm or R(R(sorted_perm[::-1]))
    moves = solver(perm.copy(), **solver_kwargs).solve()

    print('Solution = {}'.format(''.join(moves)))
    print('n={}, len(moves)={}, diff with optimal={}'.format(
        n,
        len(moves),
        len(moves) - n * (n - 1) / 2)
    )

    visualize_solution(perm, moves, start_pos)


In [2]:
# misc.py

import numpy as np
# from logger import log

def argmax(a, key=None, *args, **kwargs):
    key = key or (lambda x: x)
    keys = [key(x) for x in a]
    maximum = max(keys, *args, **kwargs)
    optimums = [i for i in range(len(a)) if keys[i] == maximum]

    # if len(optimums) > 1:
    #     log('Several optimums = {}'.format(optimums))
    return np.random.choice(optimums)
    # return optimums[0]

def argmin(a, *args, **kwargs):
    maximum = min(a, *args, **kwargs)
    return [i for i in range(len(a)) if a[i] == maximum][0]

# Solver and ProfileInversionsSolver base classes

In [3]:
# solver.py

# from logger import log, print_permutation
import numpy as np

class Solver:

    def __init__(self, perm):
        self.perm = perm
        self.n = len(perm)
        self.solution = []
        self.pos = 0

    def move_left(self):
        """Moves left, updating the solution list accordingly."""
        if len(self.solution) > 0 and self.solution[-1] == 'L':
            self.solution.pop()
        else:
            self.solution.append('R')
        self.pos = (self.pos - 1 + self.n) % self.n

    def move_right(self):
        """Moves right, updating the solution list accordingly."""
        if len(self.solution) > 0 and self.solution[-1] == 'R':
            self.solution.pop()
        else:
            self.solution.append('L')
        self.pos = (self.pos + 1) % self.n

    def swap(self):
        """Swaps the current position."""
        if len(self.solution) > 0 and self.solution[-1] == 'X':
            self.solution.pop()
        else:
            self.solution.append('X')
        self.perm[self.pos], self.perm[(self.pos + 1) % self.n] = \
            self.perm[(self.pos + 1) % self.n], self.perm[self.pos]

    def move_to(self, to):
        if self.pos == to:
            return
        
        while len(self.solution) > 0 and self.solution[-1] == 'L':
            self.move_left()

        while len(self.solution) > 0 and self.solution[-1] == 'R':
            self.move_right()

        move = None
        if self.round(to - self.pos) <= self.round(self.pos - to):
            move = self.move_right
        else:
            move = self.move_left

        while self.pos != to:
            move()

    def distance(self, from_pos, to_pos):
        return min([self.round(to_pos - from_pos), self.round(from_pos - to_pos)])
    
    def signed_distance(self, from_pos, to_pos):
        return min([self.round(to_pos - from_pos), -self.round(from_pos - to_pos)], key=lambda x: np.abs(x))

    def round(self, x):
        return (x + self.n) % self.n

    def sign(self, x):
        if x > 0:
            return 1
        elif x < 0:
            return -1
        elif x == 0:
            return 0
        else:
            raise NotImplementedError()

    def swap_to(self, to):
        ### Optimize later
        if self.pos == to:
            return

        direction = None
        if (to > self.pos and ( to - self.pos <= self.n // 2 )) or \
                (to < self.pos and ( self.pos - to >= self.n // 2 )):
            direction = 'right'
        else:
            direction = 'left'

        # log('swap_to from {} to {}, directoin={}'.format(self.pos, to, direction))
        if direction == 'right':
            while self.pos != to:
                self.swap()
                self.move_right()
        else:
            while self.pos != to:
                self.move_left()
                self.swap()

        # print('end swap_to')

    def swap_in_perm(self, perm, i, j):
        perm[i], perm[j] = perm[j], perm[i]

    def swap_by_distance(self, distance):
        if distance == 0:
            return
        
        # log('swap_by_distance from {}, distance={}'.format(self.pos, distance))
        if distance > 0:
            for _ in range(distance):
                self.swap()
                self.move_right()
        else:
            for _ in range(-distance):
                self.move_left()
                self.swap()

    def is_sorted(self):
        return all([ self.perm[i] < self.perm[(i + 1) % self.n] or self.perm[i] == self.n - 1 for i in range(self.n) ])

    def solve(self, *args, **kwargs):
        """Executes the sorting process and returns the sequence of actions."""
        raise NotImplementedError()

In [4]:
# profile_solver.py

import numpy as np
import random

# from solver import Solver
# from general_solvers import GeneralSolverAnchor
# from misc import argmax, argmin
# from logger import log
# from logger import print_permutation


class ProfileInversionsSolver(Solver):

    def __init__(self, perm, start_pos=0):
        super().__init__(perm)
        self.start_pos = start_pos
    
    def compute_profile(self, perm):
        assert len(perm) == self.n

        profile = [self.compute_inversions(perm)]
        for x in perm:
            profile.append(profile[-1] + self.n - 2 * x - 1)
        
        assert profile[0] == profile[-1]

        return profile[:-1]
    
    def compute_min_inversions(self, perm):
        profile = self.compute_profile(perm)
        return min(profile)

    def find_perfect_position_distance(self, from_pos):
        '''
        Finds perfect position in terms of global inversions profile for self.perm[from_pos] element
        '''
        # TODO: can be optimized

        move_to_distance = self.signed_distance(self.pos, from_pos)
        init_profile = np.array(self.compute_self_profile())
        
        init_inv = np.min(init_profile)
        min_inv = init_inv
        min_distance = 0
        
        current_profile = init_profile.copy()
        current_perm = self.perm.copy()
        for distance in range(1, self.n):

            self.swap_in_profile(
                current_perm,
                current_profile, 
                self.round(from_pos + distance - 1),
                self.round(from_pos + distance),
            )

            inv = np.min(current_profile)

            if self.compute_gain(init_inv, inv, distance, move_to_distance, profile=init_profile) >= \
                    self.compute_gain(init_inv, min_inv, min_distance, move_to_distance, profile=init_profile):
                min_inv = inv
                min_distance = distance
        
        current_profile = init_profile.copy()
        current_perm = self.perm.copy()
        for distance in range(-1, -self.n, -1):

            self.swap_in_profile(
                current_perm,
                current_profile, 
                self.round(from_pos + distance),
                self.round(from_pos + distance + 1),
            )

            inv = np.min(current_profile)

            if self.compute_gain(init_inv, inv, distance, move_to_distance, profile=init_profile) >= \
                    self.compute_gain(init_inv, min_inv, min_distance, move_to_distance, profile=init_profile):
                min_inv = inv
                min_distance = distance

        assert np.abs(min_distance) <= self.n / 2 + 1

        gain = self.compute_gain(init_inv, min_inv, min_distance, move_to_distance, profile=init_profile)
        return min_distance, gain
    
    def solve(self, *args, **kwargs):
        self.solution = []

        moved_counts = [0] * self.n

        self.move_to(self.start_pos)

        i = 0
        while not self.is_sorted() and i < 1000:
            i += 1

            if i > 1:
                while len(self.solution) > 0 and self.solution[-1] == 'L':
                    self.move_left()

                while len(self.solution) > 0 and self.solution[-1] == 'R':
                    self.move_right()

            gains = [(x, *self.find_perfect_position_distance(x)) for x in range(self.n)]
            next = argmax(gains, key=self.gains_comparator)

            self.move_to(next)
            self.swap_by_distance(self.find_perfect_position_distance(next)[0])

        if not self.is_sorted():
            print('Iteration limit!')
            return []

        zero_pos = self.perm.index(0)
        self.move_to(zero_pos)

        return self.solution
    

    def compute_inversions(self, perm):
        assert len(perm) == self.n

        inversions_count = 0
        for i in range(self.n):
            for j in range(i):
                if perm[j] > perm[i]:
                    inversions_count += 1

        return inversions_count

    def compute_self_profile(self):
        return self.compute_profile(self.perm)

    def swap_in_profile(self, perm, profile, i, j):
        # TODO: now works only when j is 'right' to i
        assert j == self.round(i + 1)
        
        if perm[i] < perm[j]:
            profile += 1
            profile[j] += 2 * (perm[i] - perm[j])
        elif perm[i] > perm[j]:
            profile -= 1
            profile[j] += 2 * (perm[i] - perm[j])
        else:
            raise NotImplementedError()

        perm[i], perm[j] = perm[j], perm[i]

# Main solutions classes

In [5]:
class ProfileProvableSolver(ProfileInversionsSolver):
    
    def compute_gain(self, init_inv, inv, distance, move_to_distance, profile, *args, **kwargs):
        '''
        Computes gain from moving to from_pos (move_to_distance, signed) and doing distance (signed) moves and swaps
        '''

        # from_pos = self.round(self.pos + move_to_distance)
        # to_pos = self.round(self.pos + move_to_distance + distance)
        # min_pos = argmin(profile)
        profile_fine = 0
        # if self.distance(from_pos, min_pos) <= 0 or self.distance(to_pos, min_pos) <= 0:
        #     profile_fine += 1

        move_to_fine = np.abs(move_to_distance)
        if move_to_distance > 0 and distance < 0:
            move_to_fine -= 1

        distance_fine = 2 * np.abs(distance)
        if distance > 0:
            distance_fine -= 1
        
        return (2 * (init_inv - inv) - distance_fine - move_to_fine - profile_fine).item()
    
    def gains_comparator(self, x):
        '''
        Select element with the highest gain
        Skip elements which shouldn't be moved (distance = 0), even if we'll select element with gain < 0
        '''
        i, distance, gain = x
        if np.abs(distance) > 0:
            return gain
        else:
            return -1000

class ProfileBestSolver(ProfileInversionsSolver):

    def compute_gain(self, init_inv, inv, distance, move_to_distance, *args, **kwargs):
        '''
        Computes gain from doing 'distance' (signed) moves and swaps
        '''

        distance_fine = 2 * np.abs(distance)
        if distance > 0:
            distance_fine -= 1
        
        return 2 * (init_inv - inv) - distance_fine

    def gains_comparator(self, x):
        '''
        Select closest (to the left or right) element with positive gain
        If there are 2 element with same distance and gain, chooses right one
        '''
        i, distance, gain = x
        if gain > 0:
            return (-self.distance(self.pos, i), gain, self.signed_distance(self.pos, i))
        else:
            return (-1000, -1000)


class BruteSolverParameter(Solver):
    def __init__(self, perm, solver_class, **kwargs):
        super().__init__(perm)
        self.solver_class = solver_class
        self.param_name = list(kwargs.keys())[0]
        self.param_values = kwargs[self.param_name](self.n)

    def solve(self, *args, **kwargs):

        min_solution = min([
            (self.solver_class(self.perm.copy(), **{self.param_name: value}).solve(), value)
        for value in self.param_values], key=lambda x: len(x[0]))
        
        return min_solution[0]


# Tests

In [6]:
n = 25
test(n, ProfileProvableSolver);
test(n, ProfileBestSolver);
test(n, BruteSolverParameter, solver_class=ProfileProvableSolver, start_pos=range);
print('Optimal is: 2598')

Total score:  2772
Total score:  2951
Total score:  2618
Optimal is: 2598


In [7]:
n = 16
test_random(n, 100, ProfileProvableSolver)
test_random(n, 100, ProfileProvableSolver)
test_random(n, 100, BruteSolverParameter, solver_class=ProfileProvableSolver, start_pos=range)

Average score: 94.7
Average score: 93.73
Average score: 88.62


[73,
 54,
 85,
 101,
 88,
 70,
 96,
 103,
 97,
 79,
 81,
 90,
 91,
 80,
 74,
 95,
 84,
 88,
 106,
 89,
 85,
 87,
 98,
 77,
 99,
 82,
 85,
 103,
 72,
 91,
 84,
 82,
 87,
 80,
 102,
 93,
 80,
 87,
 93,
 87,
 84,
 98,
 80,
 79,
 98,
 95,
 84,
 74,
 97,
 102,
 89,
 91,
 90,
 91,
 70,
 89,
 96,
 86,
 103,
 106,
 87,
 81,
 80,
 93,
 80,
 78,
 88,
 81,
 92,
 91,
 107,
 92,
 79,
 95,
 68,
 82,
 93,
 100,
 97,
 90,
 93,
 84,
 93,
 106,
 99,
 80,
 90,
 99,
 89,
 66,
 87,
 108,
 102,
 83,
 95,
 92,
 79,
 102,
 93,
 88]

In [8]:
%%time
import pandas as  pd

test_df = pd.read_csv('test.csv')

score = 0
scores = []
solution = []
for idx in range(len(test_df)):
    permutation = test_df.permutation.values[idx]
    arr = eval('[' + test_df.permutation.values[idx] + ']')
    
    # moves = ProfileProvableSolver(arr.copy()).solve()
    # moves = BruteSolverParameter(arr.copy(), solver_class=ProfileBestSolver, start_pos=range).solve()
    moves = ProfileBestSolver(arr.copy()).solve()
    
    assert check_solution(arr.copy(), moves)
    solution.append((permutation,'.'.join(moves)))
    score += len(moves)
    scores.append(len(moves))

print('Total score: ', score)
solution_df = pd.DataFrame(solution)
solution_df.columns = ['permutation','solution']
solution_df.to_csv('submission.csv', index=False)

Total score:  961024
CPU times: user 4h 9min 51s, sys: 2min 15s, total: 4h 12min 7s
Wall time: 4h 9min 56s
